In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
import plotly.io
plotly.io.renderers.default = 'notebook_connected'

In [3]:
DOCS_LINK = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQCN9pL21lGy3XPBhKwMX7jT1_SG-Sb_4ZWZ1I0Ctd-0vNhtmH4gFKaLsV5jhz4vSjYlQ9NR_fXF_b6/pub?output=xlsx'

In [4]:
with pd.ExcelFile(DOCS_LINK) as excel_file:
        nz_df = excel_file.parse('From Briefings', index_col='Date', parse_dates=True, header=1)
        aus_df = excel_file.parse('Australia', index_col='date', parse_dates=True)
        can_df = excel_file.parse('Canada', index_col='date', parse_dates=True)
        uk_df = excel_file.parse('UK', index_col='date', parse_dates=True)
nz_df = nz_df[~nz_df.isna().all(1)]
us_df = pd.read_csv('https://covidtracking.com/api/us/daily.csv', parse_dates=True, index_col=0)


In [5]:
us_df.sort_index()['positive'].diff()

date
2020-03-04        NaN
2020-03-05       58.0
2020-03-06       47.0
2020-03-07      118.0
2020-03-08       76.0
2020-03-09      167.0
2020-03-10      194.0
2020-03-11      276.0
2020-03-12      261.0
2020-03-13      607.0
2020-03-14      528.0
2020-03-15      723.0
2020-03-16      846.0
2020-03-17     1703.0
2020-03-18     2008.0
2020-03-19     3989.0
2020-03-20     5314.0
2020-03-21     6164.0
2020-03-22     8682.0
2020-03-23    10273.0
2020-03-24     9802.0
2020-03-25    11974.0
2020-03-26    16807.0
2020-03-27    18678.0
2020-03-28    18821.0
2020-03-29    20827.0
2020-03-30    21469.0
2020-03-31    24153.0
2020-04-01    26087.0
2020-04-02    28239.0
Name: positive, dtype: float64

In [6]:
def make_country_tests_df(df, columns, millions, new_names=['confirmed', 'tests']):
    new_df = df[columns].rename(columns=dict(zip(columns,new_names))).sort_index()
    for n in new_names:
        col = new_df[n]
        new_df[n+'/1M']=col/millions
        new_df['new '+n] = col.diff()
    return new_df

In [7]:
MILLIONS = {'NZ':4.974490, 'US':329.544974, 'Aus':25.663280, 'Can': 37.976754, 'UK':66.435600}
FULL_NAMES = {'NZ':'New Zealand', 'Aus': 'Australia', 'Can': 'Canada', 'UK':'United Kingdom', 'US':'USA'}
COLUMN_NAMES={'NZ':['Confirmed', 'Tests'], 'US':['positive', 'posNeg'], 'Can':['positives', 'tests'], 'Aus': ['cases', 'tests'], 'UK': ['cases', 'tests']}
def make_tests_df(dfs_map):
    tests_df = pd.concat({FULL_NAMES.get(c, c):make_country_tests_df(df, COLUMN_NAMES[c], MILLIONS[c]) for c,df in dfs_map.items()}, sort=True, names=['country']).reset_index()
    tests_df['case rate'] = tests_df['confirmed']/tests_df['tests']
    tests_df['daily rate'] = tests_df['new confirmed']/tests_df['new tests']
    return tests_df
    
    

In [8]:
tests_df = make_tests_df({'NZ':nz_df, 'Aus':aus_df, 'Can':can_df, 'UK':uk_df, 'US':us_df })
tests_df['nice_date']=tests_df.iloc[:,1].dt.strftime('%e %b %Y')

In [9]:
tests_df

,country,level_1,confirmed,confirmed/1M,new confirmed,new tests,tests,tests/1M,case rate,daily rate,nice_date
0,New Zealand,2020-03-13,5.0,1.005128,NaN,NaN,386.0,77.595894,0.012953,NaN,13 Mar 2020
1,New Zealand,2020-03-14,6.0,1.206154,1.0,35.0,421.0,84.631791,0.014252,0.028571,14 Mar 2020
2,New Zealand,2020-03-15,8.0,1.608205,2.0,21.0,442.0,88.853330,0.018100,0.095238,15 Mar 2020
3,New Zealand,2020-03-16,8.0,1.608205,0.0,82.0,524.0,105.337432,0.015267,0.000000,16 Mar 2020
4,New Zealand,2020-03-17,11.0,2.211282,3.0,60.0,584.0,117.398970,0.018836,0.050000,17 Mar 2020
...,...,...,...,...,...,...,...,...,...,...,...
111,USA,2020-03-29,139061.0,421.978822,20827.0,95647.0,831351.0,2522.723955,0.167271,0.217749,29 Mar 2020
112,USA,2020-03-30,160530.0,487.126228,21469.0,113503.0,944854.0,2867.147353,0.169899,0.189149,30 Mar 2020
113,USA,2020-03-31,184683.0,560.418197,24153.0,104030.0,1048884.0,3182.825055,0.176076,0.232173,31 Mar 2020
114,USA,2020-04-01,210770.0,639.578864,26087.0,101076.0,1149960.0,3489.538882,0.183285,0.258093,1 Apr 2020


In [10]:
now = pd.Timestamp('now', tz='Pacific/Auckland').strftime('%e %b %Y %H:%M (%z)')

In [11]:
LABELS = {
    'case rate':'confirmed<br>case rate', 
    'tests/1M':'total tests per 1M population (10k is 1% of population)', 
    'confirmed/1M':'confirmed cases<br>per 1M population',
    'title': 'Comparison of confirmed Covid-19 cases and total tests for selected countries normalised by population<br>Last update: '+now,
}

In [12]:
fig=(px.scatter(tests_df, y='confirmed/1M', x='tests/1M',symbol='country', color='case rate', hover_data=['confirmed', 'tests', 'new confirmed', 'new tests', 'daily rate'], hover_name='nice_date', labels=LABELS, title=LABELS['title'])
    .update_traces(line_width=3, mode='markers+lines', hovertemplate='<b>%{fullData.name}</b> - %{hovertext}<br>confirmed cases=%{customdata[0]}<br>  - up %{customdata[2]}, %{y:.1f} / 1M<br>total tests=%{customdata[1]}<br>  - up %{customdata[3]}, %{x:.1f} / 1M<br>confirmed case rate=%{marker.color:.1%}<br>  - 1 day rate=%{customdata[4]:.1%}')
    .update_layout(coloraxis=dict(colorbar=dict(tickformat='.1%'), colorscale=px.colors.sequential.Sunsetdark_r), legend=dict(x=.025, y=0.95)))
fig

In [13]:
fig.write_html('../Coronavirus_testing_comparison.html', include_plotlyjs='cdn')